## Get data from Wikipedia

https://en.wikipedia.org/wiki/List_of_A24_films

In [1]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup
import re

import sys

In [2]:
sys.version

'3.10.5 (v3.10.5:f377153967, Jun  6 2022, 12:36:10) [Clang 13.0.0 (clang-1300.0.29.30)]'

In [3]:
wiki_films_url = 'https://en.wikipedia.org/wiki/List_of_A24_films'

page = rq.get(url=wiki_films_url)
soup = BeautifulSoup(page.text, 'html.parser')

for h in soup.find_all('h3'):
    h.name = 'h2'

In [4]:
tables = soup.find_all('table')

In [5]:
table_data = []

In [6]:
for t in tables:
    
    page_section = t.find_previous('h2').text.removesuffix('[edit]')
    t_header = t.find_all('tr')[0]
    t_header_cols = [th.text.strip().removesuffix('[a]') for th in t_header.find_all('th')]
    
    t_header_cols_fmt = []
    
    for col in t_header_cols:
        col = col.lower()
        col = col.replace(r' ', '_')
        col = re.sub("\(.*\)|\[.*\]|\.", "", col)
        t_header_cols_fmt.append(col)
        
    data = {}
    
    for row_ix, row in enumerate(t.find_all('tr')[1:]):
        data[row_ix] = {}
        
        for th in row.find_all('th'):
            th.name = 'td'
                        
        for col_ix, col in enumerate(row.find_all('td')):
            
            text_fmt = re.sub("\[.*\]|", "", col.text)
            text_fmt = text_fmt.strip()

            data[row_ix][t_header_cols_fmt[col_ix]] = text_fmt
            
    table_data.append((page_section, data))


In [7]:
valid_sections = ['2010s', '2020s', 'Dated films'] 

In [8]:
df = pd.concat([pd.DataFrame.from_dict(t[1], orient='index') for t in table_data if t[0] in valid_sections], ignore_index=True)

In [9]:
df

,release_date,title,director,synopsis,notes,ref
0,"February 8, 2013",A Glimpse Inside the Mind of Charles Swan III,Roman Coppola,Charles' life falls apart when his girlfriend ...,,
1,"March 15, 2013",Ginger & Rosa,Sally Potter,"In 1960s London, a teenager's best friend fall...",,
2,"March 15, 2013",Spring Breakers,Harmony Korine,"During their spring break in Florida, four col...",,
3,"June 14, 2013",The Bling Ring,Sofia Coppola,A group of fame-obsessed teenagers known as th...,,
4,"August 2, 2013",The Spectacular Now,James Ponsoldt,An encounter between high schoolers Sutton and...,,
...,...,...,...,...,...,...
152,"January 12, 2024",My Mercury,"Joelle Chesselet, Philippa Ehrlich",A passionate conservationist makes a cruel pac...,"Also produced by A24, distributed by Amazon Pr...",
153,"January 23, 2024",Open Wide,Sara Goldblatt,John Mew thinks we're all ugly. And modern liv...,Also produced by A24; distributed by Netflix,
154,"March 1, 2024",Problemista,Julio Torres,A struggling aspiring toy designer from El Sal...,Also produced by A24,
155,"March 8, 2024",Love Lies Bleeding,Rose Glass,Reclusive gym manager Lou falls hard for Jacki...,Also produced by A24,


In [10]:
df = df.drop(labels=["ref"], axis=1)

In [23]:
df.release_date = pd.to_datetime(df.release_date)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   release_date  157 non-null    datetime64[ns]
 1   title         157 non-null    object        
 2   director      157 non-null    object        
 3   synopsis      157 non-null    object        
 4   notes         157 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 6.3+ KB


## Write to BQ

In [26]:
from google.cloud import bigquery

In [27]:
schema = []

for col, dtype in zip(df.columns, df.dtypes):

    if col == 'release_date':
        bq_dtype = bigquery.enums.SqlTypeNames.DATE
    else:
        bq_dtype = bigquery.enums.SqlTypeNames.STRING
    
    bq_col = bigquery.SchemaField(col, bq_dtype)
    
    schema.append(bq_col)

In [28]:
schema

[SchemaField('release_date', 'DATE', 'NULLABLE', None, None, (), None),
 SchemaField('title', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('director', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('synopsis', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('notes', 'STRING', 'NULLABLE', None, None, (), None)]

In [29]:
# Create a new Google BigQuery client using Google Cloud Platform project
# defaults.
client = bigquery.Client()

# Prepare a reference to a new dataset for storing the query results.
dataset_id = "wikipedia"
table_name = 'list_of_A24_films'

table_id = f"{dataset_id}.{table_name}"

In [30]:
client.project

'a24-portfolio'

In [31]:
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    
    schema=schema,
    
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.

job.result()  # Wait for the job to complete.

LoadJob<project=a24-portfolio, location=US, id=5fbe32f0-743c-4ed8-8a82-8c648dc07b1a>